In [4]:
#Importing Libraries
import pandas as pd
import numpy as np
import time
import requests
import lxml
import json
import string
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [5]:
#Collecting Information of number of colleges
dic = {}
for i in list(string.ascii_uppercase):
    url = "http://www.studyguideindia.com/Colleges/default.asp?cat={}".format(i)
    html = requests.get(url).text
    soup = BeautifulSoup(html, "lxml")
    body = soup.find("div", class_ = "PR5").find("table")
    main = body.find("h3", class_ = "subheading").text.replace(" Search Results - (", "").replace(") Colleges found", "")
    dic[i] = int(main)

In [6]:
#Collecting Links of the colleges
dic = {'X': 9,'Z': 14}
chromedriver = r"C:\Users\jaske\Downloads\chromedriver_win32\chromedriver"
l = []
for k,v in dic.items():
    url = "http://www.studyguideindia.com/Colleges/default.asp?cat={}"
    driver = webdriver.Chrome(chromedriver)
    driver.get(url.format(k))
    
    if(v > 30):
        for j in range(2, (v//30)+2):
            try:
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")
                table = soup.find("table", class_ = "clg-listing")
                links = table.find_all("a", href = True)
                for link in links:
                    l.append(link["href"])
                time.sleep(2)
                driver.find_element_by_link_text(str(j)).click()
            except Exception as e:
                pass
    else:
        try:
            html = driver.page_source
            soup = BeautifulSoup(html, "lxml")
            table = soup.find("table", class_ = "clg-listing")
            links = table.find_all("a", href = True)
            for link in links:
                l.append(link["href"])
        except Exception as e:
                pass
    driver.close()

In [7]:
#Cleaning the Data
df = pd.DataFrame()
row = 0

for i in l:
    try:
        m = []
        html = requests.get(i).text
        soup = BeautifulSoup(html, "lxml")
        body = soup.find("div", {"id":"college_details-new"}).find("div", {"id":"clg_dtl"})
        main = body.table.tbody
        content = main.find_all("td")
        for con in content:
            m.append(con.text.replace("\n","").replace("\r","").replace("\t","").replace("  ",""))
        dic_json = {m[j]: m[j + 1] for j in range(0, len(m), 2)}
        df = df.append(dic_json, ignore_index = True)
    except Exception as e:
        pass

#Saving to CSV
df.to_csv(r"C:\Users\jaske\OneDrive\Desktop\data.csv")